In [186]:
import pandas as pd
import yfinance as yf
from datetime import datetime as dt
from rapidfuzz import fuzz
pd.set_option('future.no_silent_downcasting', True)

# Get WACC

In [3]:
def get_risk_free_rate():
    """
    Get the current 10-year Treasury yield (risk-free rate) from an API.
    """
    try:
        tnx = yf.Ticker("^TNX")
        current_yield = tnx.history(period="1d")['Close'].iloc[-1] * 0.01
        return round(current_yield, 3)
    except:
        return 0.04
        
def get_wacc(ticker):
    stock= yf.Ticker(ticker)
    #Get Market Cap
    market_cap = stock.info.get('marketCap', 0)

    #Get Total Debt
    total_debt = stock.balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in stock.balance_sheet.index else 0

    # Get Beta
    beta = stock.info.get('beta', 1)

    # Get Interest Expense & Total Revenue to estimate Cost of Debt (Rd)
    interest_expense = stock.financials.loc['Interest Expense'].iloc[0] if 'Interest Expense' in stock.financials.index else 0
    cost_of_debt = abs(interest_expense / total_debt) if total_debt and not pd.isna(interest_expense) else 0.05 # Default 5% if no data

    # Get Tax Rate
    tax_rate = 0.21  # Default to 21%

    # Get Risk-Free Rate and Market Return (estimated ~8%)
    risk_free_rate = get_risk_free_rate()
    market_return = 0.08

    #Calculate Cost of Equity (Re) using CAPM
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
    
    # Total Value (V = E + D)
    total_value = market_cap + total_debt

    # Calculate WACC
    wacc = (market_cap / total_value * cost_of_equity) + (total_debt / total_value * cost_of_debt * (1 - tax_rate))

    return round(wacc, 4), round(beta, 2), cost_of_debt, tax_rate, risk_free_rate, market_return, cost_of_equity, total_debt, market_cap, total_value
    
#past 4 year revenue growth
def get_revenue_growth(ticker):
    stock= yf.Ticker(ticker)
    revenue= stock.income_stmt.loc['Total Revenue'].dropna() /1e9
    revenue = revenue.sort_index()

    growth_rates = revenue.pct_change()
    avg_growth = pd.Series(growth_rates[1:]).mean()

    return revenue, avg_growth
    
#5 year revenue projection
def get_revenue_projection(ticker, growth_rate):
    stock= yf.Ticker(ticker)
    revenue_projection = []
    projection_years=[]
    last_year = stock.income_stmt.columns[0].year
    last_year_revenue= stock.income_stmt.loc['Total Revenue'].iloc[0]/1e9
    
    for x in range(5):
        next_year_revenue = (last_year_revenue * (1 + growth_rate))
        revenue_projection.append(round(next_year_revenue, 2))
        last_year_revenue = next_year_revenue
    
        projection_years.append(last_year + (x + 1))
        
    return pd.Series(revenue_projection, projection_years) 

def get_ebit_margin(ticker):
    stock= yf.Ticker(ticker)
    if 'Operating Income' in stock.income_stmt.index:
        operating_income = stock.income_stmt.loc['Operating Income'].dropna()
        revenue = stock.income_stmt.loc['Total Revenue'].dropna()
        operating_margin = operating_income/ revenue
        
        return operating_income/1e9, operating_margin,  operating_margin.mean()
        
    else:
        print("Operating Income not found")
        
def get_ebit_projection(revenue_projection, ebit_margin):
    ebit_projection = []
    
    for revenue in revenue_projection:
        ebit = revenue * ebit_margin
        ebit_projection.append(round(ebit, 2))
    return pd.Series(ebit_projection, revenue_projection.index) 

def get_depreciation_and_amortization(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Depreciation And Amortization' in stock.cashflow.index:
        da = stock.cashflow.loc['Depreciation And Amortization'].dropna()/1e9
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        da_rate = (da/revenue).mean()
        da = change_timestamp_to_year(da)
        return da, da_rate * projection
    else:
        print("Depreciation & Amortization not found")

def get_fcf_pv(fcf_projection, wacc):
    discounted = []
    
    for t, fcf in enumerate(fcf_projection, start=1):
        pv = fcf/((1 + wacc) ** t)
        discounted.append(pv)
        
    return pd.Series(discounted, fcf_projection.index)  

def get_terminal_value(fcf_projection, wacc):
    terminal_growth = 0.02
    final_year_fcf = fcf_projection.iloc[-1]
    
    terminal_value = (final_year_fcf * (1 + terminal_growth))/(wacc - terminal_growth)
    
    return terminal_value

def get_pv_tv(tv, wacc):
    n = 5
    pv_tv= tv/((1 + wacc) ** n)

    return pv_tv

def get_enterprise_value(pv_fcf, pv_tv):
    pv_fcf_sum = pv_fcf.sum()
    
    ev = pv_fcf_sum + pv_tv

    return ev
def get_net_debt(ticker):  
    stock= yf.Ticker(ticker)
    cash = stock.balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'].iloc[0] 
    debt = stock.balance_sheet.loc['Long Term Debt'].iloc[0] 
    net_debt = debt - cash 

    return cash/1e9, debt/1e9, net_debt/1e9

def get_equity_value(ev, net_debt):
    return ev - net_debt

def get_implied_share_price(ticker, equity_value):
    stock= yf.Ticker(ticker)
    share_outstanding = stock.info['sharesOutstanding']/1e9
    per_share_value = equity_value / share_outstanding

    return share_outstanding, per_share_value

def change_timestamp_to_year(value):
    return value.groupby(value.index.year).sum()

def get_total_cash(ticker, past_revenue):
    stock= yf.Ticker(ticker)
    total_cash = stock.balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'].dropna() /1e9
    total_cash = change_timestamp_to_year(total_cash)
    cash_margin = (total_cash / past_revenue).mean()

    return cash_margin, total_cash

def get_receivables(ticker, past_recenue):
    stock= yf.Ticker(ticker)
    receivables = stock.balance_sheet.loc['Receivables'].dropna() /1e9
    receivables = change_timestamp_to_year(receivables)
    receivables_margin = (receivables/ past_revenue).mean()

    return receivables_margin, receivables
    
def get_inventory(ticker, past_recenue):
    stock= yf.Ticker(ticker)
    inventory = stock.balance_sheet.loc['Inventory'].dropna() /1e9
    inventory = change_timestamp_to_year(inventory)
    inventory_margin = (inventory/ past_revenue).mean()

    return inventory_margin, inventory
    
def get_payables(ticker, past_recenue):
    stock= yf.Ticker(ticker)
    payables = stock.balance_sheet.loc['Payables'].dropna() /1e9
    payables = change_timestamp_to_year(payables)
    payables_margin = (payables/ past_revenue).mean()

    return payables_margin, payables

def get_capEx(ticker, past_revenue):
    stock= yf.Ticker(ticker)
    
    if 'Capital Expenditure' in stock.cashflow.index:
        capex = abs(stock.cashflow.loc['Capital Expenditure'].dropna()/1e9)
        capex = change_timestamp_to_year(capex)
        capex_margin = (capex/past_revenue).mean()
        
        return capex_margin, capex
    else:
        print("Capital Expenditure not found")
def get_ufcf_pv(ufcf, wacc): 
    pv_ufcf = []
    
    for t, fcf in enumerate(ufcf, start=1):
        pv = fcf/((1 + wacc) ** t)
        pv_ufcf.append(pv)

    return pd.Series(pv_ufcf, ufcf.index) 

In [4]:
ticker = "AAPL"

In [5]:
past_revenue, rate = get_revenue_growth(ticker)

In [6]:
wacc, beta, cost_of_debt, tax_rate, risk_free_rate, market_return, cost_of_equity, total_debt, market_cap, total_value = get_wacc(ticker)

In [7]:
projection = get_revenue_projection(ticker=ticker, growth_rate= rate)

In [8]:
past_ebit, past_ebit_margin, ebit_margin= get_ebit_margin(ticker)

In [9]:
ebit = get_ebit_projection(projection, ebit_margin)

In [10]:
past_da, da_estimite = get_depreciation_and_amortization(ticker, projection)

# Operating Data

In [12]:
past_revenue = change_timestamp_to_year(past_revenue)

In [13]:
revenue_output = pd.concat([past_revenue, projection])

In [14]:
revenue_pct_change = revenue_output.pct_change()

In [15]:
past_ebit = change_timestamp_to_year(past_ebit)

In [16]:
ebit_output = pd.concat([past_ebit, ebit])

In [17]:
ebit_margin_output = ebit_output / revenue_output

In [18]:
da_output = pd.concat([past_da, da_estimite])

In [19]:
da_rate_output = da_output / revenue_output

In [20]:
pd.DataFrame(data=[revenue_output, revenue_pct_change, ebit_output, ebit_margin_output, da_output, da_rate_output],
            index=['Revenue','Revenue %', 'EBIT', 'EBIT %','Depreciation', 'Depreciation %'])

,2021,2022,2023,2024,2025,2026,2027,2028,2029
Revenue,365.817000,394.328000,383.285000,391.035000,400.180000,409.540000,419.110000,428.910000,438.940000
Revenue %,NaN,0.077938,-0.028005,0.020220,0.023387,0.023389,0.023368,0.023383,0.023385
EBIT,108.949000,119.437000,114.301000,123.216000,121.460000,124.300000,127.200000,130.180000,133.220000
EBIT %,0.297824,0.302887,0.298214,0.315102,0.303513,0.303511,0.303500,0.303514,0.303504
Depreciation,11.284000,11.104000,11.519000,11.445000,11.838040,12.114926,12.398024,12.687925,12.984631
Depreciation %,0.030846,0.028159,0.030053,0.029268,0.029582,0.029582,0.029582,0.029582,0.029582


# Balance Sheet

## Total Cash

In [23]:
cash_margin, past_total_cash = get_total_cash(ticker, past_revenue)

In [24]:
total_cash_estimite = projection * cash_margin

In [25]:
total_cash_output = pd.concat([past_total_cash, total_cash_estimite])

In [26]:
cash_margin_output = total_cash_output / revenue_output

## Receivable

In [28]:
receivable_margin, past_receivales = get_receivables(ticker, past_revenue)

In [29]:
receivable_estimite = projection * receivable_margin

In [30]:
receivable_output = pd.concat([past_receivales, receivable_estimite])

In [31]:
receivable_margin_output = receivable_output/ revenue_output

## Inventory

In [33]:
inventory_margin, past_inventory = get_inventory(ticker, past_revenue)

In [34]:
inventory_estimite = projection * inventory_margin

In [35]:
inventory_output = pd.concat([past_inventory, inventory_estimite])

In [36]:
inventory_margin_output = inventory_output/ revenue_output

## Payable

In [38]:
payable_margin, past_payable = get_payables(ticker, past_revenue)

In [39]:
payable_estimite = projection * payable_margin

In [40]:
payable_output = pd.concat([past_payable, payable_estimite])

In [41]:
payable_margin_output = payable_output/ revenue_output

## CAP EX 

In [43]:
capex_margin, past_capex = get_capEx(ticker, past_revenue)

In [44]:
capex_estimite = projection * capex_margin

In [45]:
capex_output = pd.concat([past_capex, capex_estimite])

In [46]:
capex_margin_output = capex_output/ revenue_output

In [47]:
pd.DataFrame(data=[total_cash_output, cash_margin_output, receivable_output, receivable_margin_output,inventory_output,inventory_margin_output,
                  payable_output, payable_margin_output, capex_output, capex_margin_output],
            index=['Total Cash', 'Total Cash %', 'Receivables', 'Receivables %', 'Inventories', 'Inventories %',
                  'Payable', 'Payable %', 'Cap Ex', 'Cap EX %'])

,2021,2022,2023,2024,2025,2026,2027,2028,2029
Total Cash,62.639000,48.304000,61.555000,65.171000,62.126822,63.579935,65.065651,66.587074,68.144203
Total Cash %,0.171230,0.122497,0.160599,0.166663,0.155247,0.155247,0.155247,0.155247,0.155247
Receivables,51.506000,60.932000,60.985000,66.243000,62.411465,63.871236,65.363759,66.892152,68.456416
Receivables %,0.140797,0.154521,0.159111,0.169404,0.155958,0.155958,0.155958,0.155958,0.155958
Inventories,6.580000,4.946000,6.331000,7.286000,6.570989,6.724681,6.881821,7.042738,7.207431
Inventories %,0.017987,0.012543,0.016518,0.018633,0.016420,0.016420,0.016420,0.016420,0.016420
Payable,54.763000,70.667000,71.430000,95.561000,75.999334,77.776919,79.594385,81.455531,83.360357
Payable %,0.149701,0.179209,0.186363,0.244380,0.189913,0.189913,0.189913,0.189913,0.189913
Cap Ex,11.085000,10.708000,10.959000,9.447000,11.025795,11.283683,11.547356,11.817367,12.093714
Cap EX %,0.030302,0.027155,0.028592,0.024159,0.027552,0.027552,0.027552,0.027552,0.027552


# Weignted Average Cost of Capital

In [49]:
metrics = [
    ('Beta', f"{beta:.2f}"),
    ('Cost of Debt', f"{cost_of_debt:.2%}"),
    ('Tax Rate', f"{tax_rate:.0%}"),
    ('Risk Free Rate', f"{risk_free_rate:.2%}"),
    ('Market Risk Premium', f"{market_return:.2%}"),
    ('Cost of Equity', f"{cost_of_equity:.2%}"),
    ('Total Debt', f"${total_debt/1e9:,.0f}"),
    ('Total Equity', f"${market_cap/1e9:,.0f}"),
    ('Total Capital', f"${total_value/1e9:,.0f}"),
    ('WACC', f"{wacc:.2%}")
]

pd.DataFrame(metrics, columns=['', 'Value']).set_index('')


,Value
,
Beta,1.18
Cost of Debt,5.00%
Tax Rate,21%
Risk Free Rate,4.40%
Market Risk Premium,8.00%
Cost of Equity,8.64%
Total Debt,$107
Total Equity,"$3,351"
Total Capital,"$3,458"


# Build Up Free Cash Flow

In [51]:
ebiat_output = ebit_output * (1 - tax_rate)

In [52]:
operating_wc = receivable_output + inventory_output - payable_output
delta_operating_wc = operating_wc.diff()

In [53]:
ufcf_output = ebiat_output + da_output - capex_output - delta_operating_wc

In [54]:
pv_ufcf_output = get_ufcf_pv(ufcf_output, wacc)

In [55]:
pd.DataFrame(data=[revenue_output, ebit_output, ebiat_output, da_output, receivable_output,
                   inventory_output, payable_output, capex_output, delta_operating_wc, ufcf_output, pv_ufcf_output],
            index=['Revenue','EBIT','EBIT After Tax' ,'Depreciation' , 'Receivables',
                   'Inventories', 'Payable', 'Cap Ex', 'Change in NWC(-)','Unlevered FCF',
                    'Present Value of FCF'])

,2021,2022,2023,2024,2025,2026,2027,2028,2029
Revenue,365.81700,394.328000,383.285000,391.035000,400.180000,409.540000,419.110000,428.910000,438.940000
EBIT,108.94900,119.437000,114.301000,123.216000,121.460000,124.300000,127.200000,130.180000,133.220000
EBIT After Tax,86.06971,94.355230,90.297790,97.340640,95.953400,98.197000,100.488000,102.842200,105.243800
Depreciation,11.28400,11.104000,11.519000,11.445000,11.838040,12.114926,12.398024,12.687925,12.984631
Receivables,51.50600,60.932000,60.985000,66.243000,62.411465,63.871236,65.363759,66.892152,68.456416
Inventories,6.58000,4.946000,6.331000,7.286000,6.570989,6.724681,6.881821,7.042738,7.207431
Payable,54.76300,70.667000,71.430000,95.561000,75.999334,77.776919,79.594385,81.455531,83.360357
Cap Ex,11.08500,10.708000,10.959000,9.447000,11.025795,11.283683,11.547356,11.817367,12.093714
Change in NWC(-),NaN,-8.112000,0.675000,-17.918000,15.015119,-0.164121,-0.167803,-0.171836,-0.175869
Unlevered FCF,NaN,102.863230,90.182790,117.256640,81.750526,99.192364,101.506471,103.884595,106.310586


# Terminal Value and Intrinsic Value

In [57]:
tv_output = get_terminal_value(ufcf_output, wacc)

In [58]:
pv_tv_output = get_pv_tv(tv_output, wacc)

In [59]:
ev_output = get_enterprise_value(pv_ufcf_output, pv_tv_output)

In [60]:
cash, debt, net_debt = get_net_debt(ticker)

In [61]:
equity_value_output = get_equity_value(ev_output, net_debt)

In [62]:
share_count, share_price = get_implied_share_price(ticker, equity_value_output)

In [121]:
metrics_2 = [
    ('Terminal Value', f"${tv_output:,.2f}B"),
    ('PV of Terminal Value', f"${pv_tv_output:,.2f}B"),
    ('Enterprise Value', f"${ev_output:,.2f}B"),
    ('Cash (+)', f"${cash:,.2f}B"),
    ('Debt (-)', f"${debt:,.2f}B"),
    ('Equity Value', f"${equity_value_output:,.2f}B"),
    ('Shares Outstanding', f"{share_count:,.0f}"),
    ('Implied Share Price', f"${share_price:,.2f}")
]

pd.DataFrame(metrics_2, columns=['', 'Value']).set_index('')


,Value
,
Terminal Value,"$1,668.26B"
PV of Terminal Value,"$1,109.47B"
Enterprise Value,"$1,629.68B"
Cash (+),$65.17B
Debt (-),$85.75B
Equity Value,"$1,609.10B"
Shares Outstanding,15
Implied Share Price,$107.12


In [84]:
df1 = pd.read_csv('sp500_dcf_failed.csv')

In [90]:
df1

,Ticker,error
0,ACN,'Inventory'
1,ADBE,'Inventory'
2,AFL,cannot unpack non-iterable NoneType object
3,ABNB,'Inventory'
4,AKAM,'Inventory'
...,...,...
186,WY,cannot unpack non-iterable NoneType object
187,WSM,'Long Term Debt'
188,WTW,'Inventory'
189,WDAY,'Inventory'


In [92]:
df1.groupby('error')['error'].count()

error
'Capital Expenditure'                           1
'Inventory'                                   106
'Long Term Debt'                               13
'Receivables'                                   6
'Total Revenue'                                 2
cannot unpack non-iterable NoneType object     63
Name: error, dtype: int64

In [94]:
df1['error'] = df1['error'].str.replace("'", "", regex=False)

In [174]:
df1[df1['error']== 'Total Revenue']

,Ticker,error
24,BRK.B,Total Revenue
31,BF.B,Total Revenue


In [96]:
long_term_debt= df1[df1['error']=='Long Term Debt']

In [298]:
ticker ='KMI'
stock= yf.Ticker(ticker)

In [230]:
income_s = stock.income_stmt
income_s[income_s.index.to_series().str.contains('Operating Income', regex= True)].index[0]

'Total Operating Income As Reported'

In [212]:
income_s = stock.income_stmt
target = 'Operating Income'
similarity_scores = income_s.index.to_series().apply(lambda x: fuzz.ratio(x, target))
# income_s.loc[similarity_scores.sort_values(ascending=False).index]
similarity_scores.sort_values(ascending=False).index[0]

'Operating Revenue'

In [214]:
income_s = stock.balance_sheet
target = 'Long Term Debt'
similarity_scores = income_s.index.to_series().apply(lambda x: fuzz.ratio(x, target))
# income_s.loc[similarity_scores.sort_values(ascending=False).index]
similarity_scores.sort_values(ascending=False).index[0]

'Long Term Debt'

In [158]:
tickers = long_term_debt['Ticker']
debt_df = []
failed = 0

for ticker in tickers:
    try:
        stock= yf.Ticker(ticker)
        bs = stock.balance_sheet
        line_item = bs[bs.index.to_series().str.contains('Long Term Debt', regex=True)].index[0]
        debt = bs.loc[line_item].iloc[0] / 1e9
        if pd.isna(debt):
            debt = 1
        debt_df.append({'Name': ticker, 'Debt': debt})
    except Exception as e:
        failed += 1

In [160]:
failed

0

In [162]:
df4= pd.DataFrame(debt_df)

In [124]:
df4['Debt'].isna()

0     False
1      True
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12    False
Name: Debt, dtype: bool

In [164]:
df4

,Name,Debt
0,ALGN,0.088214
1,ANET,0.000000
2,CMG,4.262782
3,DECK,0.213298
4,GRMN,0.134886
5,INCY,0.033542
6,ISRG,0.000000
7,LULU,1.300637
8,MRNA,0.710000
9,MPWR,0.012974


In [184]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [300]:
stock.cashflow.index

Index(['Free Cash Flow', 'Repurchase Of Capital Stock', 'Repayment Of Debt',
       'Issuance Of Debt', 'Capital Expenditure',
       'Interest Paid Supplemental Data', 'Income Tax Paid Supplemental Data',
       'End Cash Position', 'Beginning Cash Position',
       'Effect Of Exchange Rate Changes', 'Changes In Cash',
       'Financing Cash Flow', 'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Preferred Stock Dividend Paid', 'Common Stock Dividend Paid',
       'Net Common Stock Issuance', 'Common Stock Payments',
       'Net Issuance Payments Of Debt', 'Net Long Term Debt Issuance',
       'Long Term Debt Payments', 'Long Term Debt Issuance',
       'Investing Cash Flow', 'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Dividends Received Cfi',
       'Net Investment Purchase And Sale', 'Purchase Of Investment',
       'Net Business Purchase And Sale', 'Sale Of Business',


In [302]:
cf = stock.cashflow.index
target = 'Depreciation And Amortization'

In [260]:
income_s = stock.income_stmt.index
target2 = 'Operating Income'

In [242]:
import difflib

In [282]:
def get_best_match_index(index_list, target_name, cutoff=0.6):
    # Check for exact match first
    if target_name in index_list:
        return target_name
    else:
        # Find closest matches using difflib
        close_matches = difflib.get_close_matches(target_name, index_list, n=1, cutoff=cutoff)
        if close_matches:
            return close_matches[0]
        else:
            return None 

In [304]:
get_best_match_index(cf, target)

'Depreciation Amortization Depletion'

In [286]:
get_best_match_index(income_s, target2)

'Net Income'

In [262]:
income_s

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Net Income From Continuing Operation Net Minority Interest', 'EBIT',
       'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Income Expense', 'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Other Operating Expenses',
       'Total Revenue', 'Operating Revenue', 'Loss Adjustment Expense',
       'Net Policyholder Benefits And Claims', 'Policyholder Benefits Gross',
       'Policyholder Benefits Ceded'],
      dtype='object')

In [306]:
stock.info

{'address1': '1001 Louisiana Street',
 'address2': 'Suite 1000',
 'city': 'Houston',
 'state': 'TX',
 'zip': '77002',
 'country': 'United States',
 'phone': '713-369-9000',
 'website': 'https://www.kindermorgan.com',
 'industry': 'Oil & Gas Midstream',
 'industryKey': 'oil-gas-midstream',
 'industryDisp': 'Oil & Gas Midstream',
 'sector': 'Energy',
 'sectorKey': 'energy',
 'sectorDisp': 'Energy',
 'longBusinessSummary': 'Kinder Morgan, Inc. operates as an energy infrastructure company primarily in North America. The company operates through Natural Gas Pipelines, Products Pipelines, Terminals, and CO2 segments. The Natural Gas Pipelines segment owns and operates interstate and intrastate natural gas pipeline, and storage systems; natural gas gathering systems and natural gas processing and treating facilities; natural gas liquids fractionation facilities and transportation systems; and liquefied natural gas gasification, liquefaction, and storage facilities. The Products Pipelines segm